1. Creating a relatedness matrix A.

In [7]:
pedigree = read.table("pedigree.csv", sep = ";")

In [9]:
pedigreeNew = c(rbind(pedigree$V3, pedigree$V2, pedigree$V1))
length(pedigreeNew)
pedigreeNew = data.frame(pedigreeNew)
colnames(pedigreeNew) = 'id'
pedigreeNew = unique(pedigreeNew)
dim(pedigreeNew)

[1] 537

[1] 499   1

In [10]:
colnames(pedigree) = c('id', 'sire', 'dam', 'birthYear', 'breed')

In [11]:
pedigreeNew = merge(pedigreeNew, pedigree, by = 'id', all.x = TRUE)
head(pedigreeNew)
dim(pedigreeNew)
summary(pedigreeNew)

,id,sire,dam,birthYear,breed
,<chr>,<chr>,<chr>,<int>,<int>
1,cow1,sire98,dam114,2005,6
2,cow10,sire117,dam140,2005,5
3,cow100,sire92,dam107,2006,4
4,cow101,sire61,dam68,2005,6
5,cow102,sire18,dam18,2005,5
6,cow103,sire25,dam25,2005,2


[1] 499   5

      id                sire               dam              birthYear   
 Length:499         Length:499         Length:499         Min.   :2003  
 Class :character   Class :character   Class :character   1st Qu.:2005  
 Mode  :character   Mode  :character   Mode  :character   Median :2006  
                                                          Mean   :2007  
                                                          3rd Qu.:2006  
                                                          Max.   :2015  
                                                          NA's   :320   
     breed      
 Min.   :1.000  
 1st Qu.:2.000  
 Median :4.000  
 Mean   :3.821  
 3rd Qu.:6.000  
 Max.   :6.000  
 NA's   :320    

In [12]:
pedigreeNew[is.na(pedigreeNew)] = 0
summary(pedigreeNew)

      id                sire               dam              birthYear     
 Length:499         Length:499         Length:499         Min.   :   0.0  
 Class :character   Class :character   Class :character   1st Qu.:   0.0  
 Mode  :character   Mode  :character   Mode  :character   Median :   0.0  
                                                          Mean   : 719.8  
                                                          3rd Qu.:2005.0  
                                                          Max.   :2015.0  
     breed      
 Min.   :0.000  
 1st Qu.:0.000  
 Median :0.000  
 Mean   :1.371  
 3rd Qu.:3.000  
 Max.   :6.000  

In [13]:
pedigreeNew = pedigreeNew[order(pedigreeNew$birthYear), ]
head(pedigreeNew)

,id,sire,dam,birthYear,breed
,<chr>,<chr>,<chr>,<dbl>,<dbl>
180,dam1,0,0,0,0
181,dam10,0,0,0,0
182,dam100,0,0,0,0
183,dam101,0,0,0,0
184,dam102,0,0,0,0
185,dam103,0,0,0,0


In [14]:
install.packages("AGHmatrix")

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

also installing the dependency ‘zoo’




In [15]:
library(AGHmatrix)

In [16]:
A = Amatrix(pedigreeNew[, 1:3])

Verifying conflicting data... 
Organizing data... 
Your data was chronologically organized with success. 
Constructing matrix A using ploidy = 2 
Completed! Time = 9e-04  minutes 


In [18]:
pedigreeNew$idNew = 1:nrow(pedigreeNew)
head(pedigreeNew)

,id,sire,dam,birthYear,breed,idNew
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>
180,dam1,0,0,0,0,1
181,dam10,0,0,0,0,2
182,dam100,0,0,0,0,3
183,dam101,0,0,0,0,4
184,dam102,0,0,0,0,5
185,dam103,0,0,0,0,6


In [73]:
(n_ped = nrow(A))

[1] 499

2. Generating IFL Phenotype

In [46]:
library(data.table)
phenotype = read.table("http://theta.edu.pl/wp-content/uploads/2023/01/laktacje.csv", sep = ";", header = TRUE)
head(phenotype)

,cowId,success,calvingDate,inseminationDate
,<chr>,<int>,<chr>,<chr>
1,cow1,0,25.08.2012,15.09.2011
2,cow1,1,25.08.2012,06.11.2011
3,cow2,0,26.02.2013,04.06.2011
4,cow2,0,26.02.2013,21.07.2011
5,cow2,0,26.02.2013,21.11.2011
6,cow2,0,26.02.2013,13.01.2012


In [47]:
phenotype$date = as.Date(phenotype$inseminationDate, format = "%d.%m.%Y")
head(phenotype)

,cowId,success,calvingDate,inseminationDate,date
,<chr>,<int>,<chr>,<chr>,<date>
1,cow1,0,25.08.2012,15.09.2011,2011-09-15
2,cow1,1,25.08.2012,06.11.2011,2011-11-06
3,cow2,0,26.02.2013,04.06.2011,2011-06-04
4,cow2,0,26.02.2013,21.07.2011,2011-07-21
5,cow2,0,26.02.2013,21.11.2011,2011-11-21
6,cow2,0,26.02.2013,13.01.2012,2012-01-13


In [50]:
phenotype$date = as.Date(phenotype$date)
phenotype = as.data.table(phenotype)

In [51]:
minDate = phenotype[, .(minDate = min(date)), by = cowId ]
maxDate = phenotype[, .(maxDate = max(date)), by = cowId ]
IFL = merge(minDate,  maxDate, by = 'cowId', all.x = TRUE)

In [52]:
summary(IFL)

    cowId              minDate              maxDate          
 Length:179         Min.   :2010-03-01   Min.   :2010-03-01  
 Class :character   1st Qu.:2011-05-27   1st Qu.:2011-06-24  
 Mode  :character   Median :2012-01-10   Median :2012-02-21  
                    Mean   :2013-04-25   Mean   :2013-06-12  
                    3rd Qu.:2013-11-22   3rd Qu.:2014-01-05  
                    Max.   :2020-12-20   Max.   :2020-12-20  

In [53]:
IFL$IFL = as.integer(IFL$maxDate - IFL$minDate)
summary(IFL)

    cowId              minDate              maxDate                IFL        
 Length:179         Min.   :2010-03-01   Min.   :2010-03-01   Min.   :  0.00  
 Class :character   1st Qu.:2011-05-27   1st Qu.:2011-06-24   1st Qu.:  0.00  
 Mode  :character   Median :2012-01-10   Median :2012-02-21   Median :  0.00  
                    Mean   :2013-04-25   Mean   :2013-06-12   Mean   : 48.35  
                    3rd Qu.:2013-11-22   3rd Qu.:2014-01-05   3rd Qu.: 64.00  
                    Max.   :2020-12-20   Max.   :2020-12-20   Max.   :477.00  

In [54]:
IFL = IFL[, c(1, 4)]

In [55]:
IFL = merge(IFL, pedigreeNew, by.x = 'cowId', by.y = 'id', all.x = TRUE)
head(IFL)
dim(IFL)
summary(IFL)

cowId,IFL,sire,dam,birthYear,breed,idNew
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<int>
cow1,52,sire98,dam114,2005,6,333
cow10,0,sire117,dam140,2005,5,334
cow100,127,sire92,dam107,2006,4,408
cow101,0,sire61,dam68,2005,6,335
cow102,0,sire18,dam18,2005,5,336
cow103,0,sire25,dam25,2005,2,337


[1] 179   7

    cowId                IFL             sire               dam           
 Length:179         Min.   :  0.00   Length:179         Length:179        
 Class :character   1st Qu.:  0.00   Class :character   Class :character  
 Mode  :character   Median :  0.00   Mode  :character   Mode  :character  
                    Mean   : 48.35                                        
                    3rd Qu.: 64.00                                        
                    Max.   :477.00                                        
   birthYear        breed           idNew      
 Min.   :2003   Min.   :1.000   Min.   :321.0  
 1st Qu.:2005   1st Qu.:2.000   1st Qu.:365.5  
 Median :2006   Median :4.000   Median :410.0  
 Mean   :2007   Mean   :3.821   Mean   :410.0  
 3rd Qu.:2006   3rd Qu.:6.000   3rd Qu.:454.5  
 Max.   :2015   Max.   :6.000   Max.   :499.0  

In [56]:
IFL = IFL[, -c(3, 4)]

In [74]:
IFL = data.frame(IFL)
IFL = IFL[order(IFL$idNew),]
(n_ph = nrow(IFL))

[1] 179

3. Creating Design Matrix for Fixed Effects

In [75]:
X1 = model.matrix(~ factor(IFL$birthYear) - 1)
X2 = model.matrix(~ factor(IFL$breed) - 1)
X = cbind(X1, X2)

(n_fix = ncol(X))

[1] 19

Creating Matrix for Random Effects

In [76]:
Z = matrix(0, n_ph, n_ped)
dim(Z)
I = diag(n_ph)
Z[1:n_ph, (n_ped - n_ph + 1):n_ped] = I

[1] 179 499

5. Solving the System of Equations for the Mixed Model

In [77]:
library(MASS)

mme = function(y, X, Z, A, sigma_a, sigma_e) {
    alpha = sigma_e / sigma_a
    invA = ginv(A)
    C = rbind(cbind(t(X)%*%X, t(X)%*%Z),
              cbind(t(Z)%*%X, t(Z)%*%Z+invA*alpha))
    rhs = rbind(t(X)%*%y, t(Z)%*%y)
    invC = ginv(C)
    estimators = invC%*%rhs
    list(mC = C, est = estimators)
}

y = as.matrix(IFL$IFL)

sigma_alpha = 235.55
sigma_error = 3632.90

res = mme(y, X, Z, A, sigma_alpha, sigma_error)

6. Estimation Accuracy

In [64]:
C = res$mC
invC = ginv(C)
(k = ncol(X))
(n = nrow(C))

[1] 19

[1] 518

In [71]:
invC22 = invC[(k+1):n, (k+1):n]
alpha = sigma_error / sigma_alpha
r2 = diag(1 - invC22*alpha)
r2[r2 < 0] = 0
r = sqrt(r2)
summary(r)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.1193  0.1219  0.1616  0.2376  0.2958 

7. Odziedziczalność

In [37]:
(h2 = sigma_alpha / (sigma_alpha + sigma_error))

[1] 0.06089002

8. Istotność efektów stałych

In [68]:
G = A*sigma_alpha
R = diag(nrow(IFL))*sigma_error
V = Z%*%G%*%t(Z) + R

varB = ginv(t(X)%*%ginv(V)%*%X)
seB = sqrt(diag(varB))

testWalda = res$est[1:k] / seB
p_value = 2*pnorm(abs(testWalda), lower.tail = FALSE)

which(p_value < 0.05)

[1]  2  4 16 17 19